In [1]:
%matplotlib notebook
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.io.fits import getheader
from utilities import redshifts_from_papers

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

/home/boada/Projects/swiftXRT/paper/utilities.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def load_detcats(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_vtp.detect'):
            data_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            if len(data_part) > 0:
                data_part['Field'] = field
                try:
                    hdr = getheader(f'{outpath}/{field}/{field}_vtp_high.detect', ext=1)
                except FileNotFoundError:
                    continue
                data_part['Field_EXP'] = hdr['EXPOSURE']
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data


def load_MCMCfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_mcmcfits.txt'):
            try:
                fit_part = Table.read(f'{outpath}/{field}/{field}_mcmcfits.txt', format='ascii', header_start=0)
            except :
                print(field)
                raise (FileNotFoundError)
            src_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            
            fit_part.rename_column('ID', 'INDEX')
            try:
                data_part = join(fit_part, src_part, keys='INDEX', join_type='inner')
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

def load_spectralfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/spec_files/{field}_xspec.results'):
            try:
                data_part = Table.read(f'{outpath}/{field}/spec_files/{field}_xspec.results', 
                                       format='ascii', header_start=0)
                if len(data_part) < 1:
                    continue
                
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

In [3]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')
data = redshifts_from_papers(data)

datapath = '../data'
data.replace(-1, np.nan, inplace=True)

fields = [n.replace(' ', '_') for n in data['NAME']]

fits = load_MCMCfits(datapath, fields) # not currently used, but gives access to beta-model params
cats = load_detcats(datapath, fields)
specs = load_spectralfits(datapath, fields)

# check to see which fields have XRT data
field_mask = [True if os.path.isdir(f'{datapath}/{n.replace(" ", "_")}') else False for n in data['NAME']]
                                    
data['SWIFT'] = False
data['SWIFT'] = field_mask

/home/boada/Projects/swiftXRT/paper/utilities.py:215: TableReplaceWarning: converted column 'Nspec' from integer to float
  data_barrena = load_extraCatalogs('barrena2018', merge=True)
/home/boada/Projects/swiftXRT/paper/utilities.py:215: TableReplaceWarning: converted column 'R' from integer to float
  data_barrena = load_extraCatalogs('barrena2018', merge=True)
/home/boada/Projects/swiftXRT/paper/utilities.py:216: TableReplaceWarning: converted column 'N' from integer to float
  data_aguado = load_extraCatalogs('aguado-barahona2019', merge=True)
/home/boada/Projects/swiftXRT/paper/utilities.py:218: TableReplaceWarning: converted column 'ID' from integer to float
  data_zohren = load_extraCatalogs('zohren2019', merge=True)


In [4]:
# add more columns -- don't think I really need these in the end.
# data['XRT_exp'] = np.nan
# data['N_H'] = np.nan

In [5]:
cats.remove_column('R')
df_cats = cats.to_pandas()
df_cats.Field = df_cats.Field.str.replace('_', ' ')

df_specs = specs.to_pandas()
df_specs.FIELD = df_specs.FIELD.str.replace('_', ' ')

fits.remove_column('R')
df_fits = fits.to_pandas()

In [6]:
# df_fits

In [7]:
# df_cats

In [8]:
# df_specs

In [9]:
df_data = data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC', 'REDSHIFT']]

In [10]:
# df_data

In [11]:
# merge the catalog information to the main "data" frame
df_m = df_data.merge(df_cats,                                                      
               how='inner',                                              
               left_on='NAME',                                          
               right_on='Field',                                           
               suffixes=('_data', '_detc'))

# merge the spectral information to this new merged frame
df_m2 = df_m.merge(df_specs,                                                      
               how='left',                                              
               left_on=['NAME', 'INDEX'],                                          
               right_on=['FIELD', 'INDEX'],                                           
               suffixes=('_data', '_detc'))

In [12]:
# df_m

In [13]:
# df_m2

In [14]:
# only select extended objects that are within 6' of the PSZ position
df_close_ext = df_m2[(df_m2.PSZ_dist < 6) & (df_m2.Extended == 1)]

In [15]:
df_close_ext.columns

Index(['NAME', 'INDEX_PSZ2', 'RA_data', 'DEC_data', 'REDSHIFT', 'INDEX',
       'RA_VTP', 'RA_ERR_VTP', 'DEC_VTP', 'DEC_ERR_VTP', 'X_VTP', 'Y_VTP',
       'X_ERR_VTP', 'Y_ERR_VTP', 'SRC_AREA', 'NET_COUNTS', 'NET_COUNTS_ERR',
       'BKG_COUNTS', 'BKG_COUNTS_ERR', 'NET_RATE', 'NET_RATE_ERR', 'BKG_RATE',
       'BKG_RATE_ERR', 'EXPTIME', 'SRC_CUTOFF', 'FSP', 'EDGE_OF_FIELD',
       'SHAPE', 'ROTANG', 'COMPONENT', 'HIGH', 'X', 'Y', 'X_ERR', 'Y_ERR',
       'RA_detc', 'DEC_detc', 'RA_ERR', 'DEC_ERR', 'PSZ_dist', 'Extended',
       'Field', 'Field_EXP', 'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG',
       'NORM_POS', 'FLUX', 'CNT_RATE_NET', 'CNT_RATE_NET_ERR',
       'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'LUM_CORR', 'CORR',
       'FIELD'],
      dtype='object')

In [16]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
        'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG', 'NORM_POS', 'FLUX', 'CNT_RATE_NET', 
        'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'LUM_CORR', 'CORR']

In [17]:
df_close_ext[df_close_ext.FLUX_CORR.isnull()][cols]

,NAME,RA_detc,DEC_detc,REDSHIFT,INDEX,PSZ_dist,EXPTIME,SN,redCHI2,DOF,...,NORM_NEG,NORM_POS,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_CORR,LUM_CORR,CORR
109,PSZ1 G292.00-43.64,38.261892,-71.275759,NaN,7,1.938189,327520.093750,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,PSZ1 G292.00-43.64,38.184324,-71.219107,NaN,9,3.472973,327940.125000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,PSZ1 G292.00-43.64,38.162681,-71.298036,NaN,11,1.291543,329085.218750,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,PSZ1 G292.00-43.64,38.157531,-71.263793,NaN,12,0.766725,322966.812500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,PSZ1 G292.00-43.64,38.120430,-71.310465,NaN,14,2.183518,325856.375000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,PSZ1 G292.00-43.64,38.117429,-71.287531,NaN,15,1.072433,334003.093750,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,PSZ1 G292.00-43.64,38.108198,-71.338280,NaN,16,3.843983,313552.562500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,PSZ1 G292.00-43.64,37.937107,-71.226898,NaN,23,5.249118,330842.937500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,PSZ1 G292.00-43.64,37.905434,-71.333463,NaN,26,5.990574,314961.250000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,PSZ1 G292.00-43.64,37.872597,-71.293228,NaN,29,5.647563,344272.687500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# calculate the errors for the flux and Luminosity
for c in ['FLUX_CORR_POSERR', 'FLUX_CORR_NEGERR', 'LUM_CORR_POSERR', 'LUM_CORR_NEGERR']:
    df_close_ext.loc[:, c] = np.nan

df_close_ext['FLUX_CORR_POSERR'] = df_close_ext['FLUX_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_POS']
df_close_ext['FLUX_CORR_NEGERR'] = df_close_ext['FLUX_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_NEG']
df_close_ext['LUM_CORR_POSERR'] = df_close_ext['LUM_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_POS']
df_close_ext['LUM_CORR_NEGERR'] = df_close_ext['LUM_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_NEG']

/home/boada/.local/lib/python3.8/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/boada/.local/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-18-8d2900c1a091>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [19]:
df_close_ext

,NAME,INDEX_PSZ2,RA_data,DEC_data,REDSHIFT,INDEX,RA_VTP,RA_ERR_VTP,DEC_VTP,DEC_ERR_VTP,...,ECF,LUM,FLUX_CORR,LUM_CORR,CORR,FIELD,FLUX_CORR_POSERR,FLUX_CORR_NEGERR,LUM_CORR_POSERR,LUM_CORR_NEGERR
0,PSZ1 G023.53-36.53,NaN,313.528542,-22.841100,0.2840,1,313.521441,0.001330,-22.811574,0.001614,...,2.157700e-11,1.302500e+44,6.225389e-13,1.414886e+44,0.086285,PSZ1 G023.53-36.53,7.536840e-14,7.536840e-14,1.712948e+43,1.712948e+43
22,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,22,256.352326,0.000212,11.435420,0.000468,...,1.990600e-11,3.394900e+41,4.282980e-15,6.131003e+41,0.805945,PSZ1 G031.41+28.75,1.104316e-15,1.104316e-15,1.580808e+41,1.580808e+41
26,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,26,256.315374,0.000186,11.451402,0.000189,...,2.041800e-11,5.230700e+41,5.323069e-15,7.619764e+41,0.456739,PSZ1 G031.41+28.75,1.146904e-15,1.146904e-15,1.641748e+41,1.641748e+41
33,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,33,256.284234,0.000194,11.378536,0.000146,...,2.257400e-11,6.697200e+41,6.398791e-15,9.159770e+41,0.367701,PSZ1 G031.41+28.75,1.233536e-15,1.233536e-15,1.765788e+41,1.765788e+41
39,PSZ1 G055.72+17.58,NaN,276.342547,27.804647,0.1940,1,276.379384,0.001091,27.741733,0.000991,...,2.077600e-11,1.333100e+44,1.447157e-12,1.431798e+44,0.074037,PSZ1 G055.72+17.58,9.555790e-14,9.555790e-14,9.454374e+42,9.454374e+42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,PSZ2 G358.34-47.31,1647.0,323.485677,-42.666426,0.2000,3,323.500210,0.000547,-42.646546,0.000361,...,2.103000e-11,9.044200e+43,9.228984e-13,9.756041e+43,0.078707,PSZ2 G358.34-47.31,4.023996e-14,4.023996e-14,4.253801e+42,4.253801e+42
913,PSZ2 G358.34-47.31,1647.0,323.485677,-42.666426,0.2000,4,323.423754,0.001739,-42.727664,0.000935,...,2.236700e-11,5.898000e+42,8.526931e-14,9.013843e+42,0.528288,PSZ2 G358.34-47.31,1.839696e-14,1.839696e-14,1.944748e+42,1.944748e+42
915,PSZ2 G358.98-67.26,1650.0,348.926557,-37.779104,0.1786,1,348.934928,0.002054,-37.793994,0.001341,...,2.088000e-11,1.456100e+44,1.879162e-12,1.557075e+44,0.069346,PSZ2 G358.98-67.26,1.840592e-13,1.840592e-13,1.525116e+43,1.525116e+43
916,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,1,303.146133,0.000335,-41.480888,0.000363,...,2.078300e-11,1.141400e+42,2.169865e-14,1.230039e+42,0.077659,PSZ2 G359.07-32.12,2.169865e-14,2.169865e-14,1.230039e+42,1.230039e+42


In [20]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
        'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG', 'NORM_POS', 'FLUX', 'CNT_RATE_NET', 
        'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'FLUX_CORR_POSERR', 'FLUX_CORR_NEGERR', 
        'LUM_CORR', 'LUM_CORR_POSERR', 'LUM_CORR_NEGERR', 'CORR']

t1 = Table.from_pandas(df_close_ext[cols])

In [21]:
t1.filled(-1.0)

NAME,RA_detc,DEC_detc,REDSHIFT,INDEX,PSZ_dist,EXPTIME,SN,redCHI2,DOF,NORM,NORM_NEG,NORM_POS,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_CORR,FLUX_CORR_POSERR,FLUX_CORR_NEGERR,LUM_CORR,LUM_CORR_POSERR,LUM_CORR_NEGERR,CORR
str18,float64,float64,float64,int64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
PSZ1 G023.53-36.53,313.5192459279641,-22.811192599451434,0.2840000092983246,1,1.8666280661228891,2716.9482,8.2599,1.321,15.0,0.00146282,0.000177098,0.000177098,5.7309e-13,0.02659,0.0265606,2.1577e-11,1.3025e+44,6.225389230644651e-13,7.536839679309186e-14,7.536839679309186e-14,1.4148858770724767e+44,1.7129479980980672e+43,1.7129479980980672e+43,0.08628474247407045
PSZ1 G031.41+28.75,256.3524228060791,11.435045572929392,0.23,22,5.466414508179784,216357.27,3.8784,0.93,1.0,5.6568e-06,1.45854e-06,1.45854e-06,2.3716e-15,6.128e-05,0.000119141,1.9906e-11,3.3949e+41,4.282979668215628e-15,1.104316427181308e-15,1.104316427181308e-15,6.13100340513798e+41,1.5808078253659223e+41,1.5808078253659223e+41,0.8059452134489913
PSZ1 G031.41+28.75,256.315810879561,11.451588212877505,0.23,26,3.141827605359394,214575.03,4.6413,0.668,3.0,8.71576e-06,1.87789e-06,1.87789e-06,3.6541e-15,0.0001279,0.000178969,2.0418e-11,5.2307e+41,5.323069335708017e-15,1.146903847149615e-15,1.146903847149615e-15,7.619763765164589e+41,1.6417476131702721e+41,1.6417476131702721e+41,0.45673882370707347
PSZ1 G031.41+28.75,256.2842552229166,11.378437505810556,0.23,33,5.074730659124593,197958.08,5.1874,0.503,3.0,1.11593e-05,2.15125e-06,2.15125e-06,4.6785e-15,0.0001845,0.00020725,2.2574e-11,6.6972e+41,6.398791126877589e-15,1.2335361009826255e-15,1.2335361009826255e-15,9.159769997846445e+41,1.7657877472482293e+41,1.7657877472482293e+41,0.3677014271406625
PSZ1 G055.72+17.58,276.3803066233068,27.741361425895942,0.1940000057220459,1,4.293797561529516,3424.389,15.1443,1.142,33.0,0.00306725,0.000202535,0.000202535,1.3474e-12,0.05858,0.0648523,2.0776e-11,1.3331e+44,1.4471571301566844e-12,9.555790018951309e-14,9.555790018951309e-14,1.4317984044915213e+44,9.45437410884963e+42,9.45437410884963e+42,0.07403675980160625
PSZ1 G057.42-10.77,304.36687269578346,16.062365346049575,0.136,1,1.9135608557965353,2917.1406,15.8061,0.6912,37.0,0.00402045,0.00025436,0.00025436,1.9085e-12,0.07181,0.0907766,2.1024e-11,8.8352e+43,2.0119497106012662e-12,1.272891164890841e-13,1.272891164890841e-13,9.314109553630759e+43,5.892715755852006e+42,5.892715755852006e+42,0.05420472130011307
PSZ1 G083.35+76.41,203.70014715336345,37.18210039979622,-1.0,1,0.9253717082144188,46343.074,-1.0,6.341,297.0,-1.0,-1.0,-1.0,3.6628e-12,0.2144,0.154388,2.3725e-11,8.8711e+43,3.711064918516037e-12,3.711064918516037e-12,3.711064918516037e-12,8.987994976151472e+43,8.987994976151472e+43,8.987994976151472e+43,0.013177055399158042
PSZ1 G083.35+76.41,203.6204264527387,37.209131244254884,-1.0,2,3.3910404629742787,52051.797,-1.0,1.55,7.0,-1.0,-1.0,-1.0,2.3894e-15,0.0002452,0.000113301,2.1089e-11,5.7871e+40,4.652337415263511e-15,4.652337415263511e-15,4.652337415263511e-15,1.1267909038198486e+41,1.1267909038198486e+41,1.1267909038198486e+41,0.9470734976410439
PSZ1 G103.50+31.36,264.23998368261863,72.56058825187006,-1.0,1,0.982848402974421,2846.7449,-1.0,2.23,4.0,-1.0,-1.0,-1.0,1.206e-13,0.009561,0.00565129,2.134e-11,2.921e+42,1.6141679232683427e-13,1.6141679232683427e-13,1.6141679232683427e-13,3.909605724599361e+42,3.909605724599361e+42,3.909605724599361e+42,0.3384476975691067


In [22]:
t1.filled(-1.0).write('results_table_full_0.5_2.0.fits', overwrite=True)